In [1]:
#!/usr/bin/env python3
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
import cityscapes_helper
import numpy as np

label_values = cityscapes_helper.get_label_info()


C:\Users\Kiko-PC\Anaconda3\envs\tensorflow\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)


In [2]:
print(label_values)
print(len(label_values))

[(0, 0, 0), (111, 74, 0), (81, 0, 81), (128, 64, 128), (244, 35, 232), (250, 170, 160), (230, 150, 140), (70, 70, 70), (102, 102, 156), (190, 153, 153), (180, 165, 180), (150, 100, 100), (150, 120, 90), (153, 153, 153), (250, 170, 30), (220, 220, 0), (107, 142, 35), (152, 251, 152), (70, 130, 180), (220, 20, 60), (255, 0, 0), (0, 0, 142), (0, 0, 70), (0, 60, 100), (0, 0, 90), (0, 0, 110), (0, 80, 100), (0, 0, 230), (119, 11, 32)]
29


In [4]:
X_train, y_train, X_val, y_val = cityscapes_helper.get_data('D:\data\leftImg8bit_trainvaltest')

Loading X_Train..
Loading Y_Train..
Loading X_Validation..
Loading Y_Validation..


In [42]:
L2_REG = 1e-5
STDEV = 1e-2
KEEP_PROB = 0.8
LEARNING_RATE = 5e-4


# L2_REG = 1e-3
# STDEV = 1e-2
# KEEP_PROB = 0.5
# LEARNING_RATE = 5e-4

In [43]:
import cv2
from collections import namedtuple
import random
import numpy as np
import os.path
import scipy.misc
import os
from glob import glob
from sklearn.utils import shuffle
import random
from cityscapes_helper import *


def flip_image(image, measurement, flip_probability=1.0):
    if random.random() <= flip_probability:
        image = cv2.flip(image, 1)
        measurement*=-1
    return image, measurement


# Randomly crop the image to a specific size. For data augmentation
def random_crop(image, label, crop_height, crop_width):
    if (image.shape[0] != label.shape[0]) or (image.shape[1] != label.shape[1]):
        raise Exception('Image and label must have the same dimensions!')

    if (crop_width <= image.shape[1]) and (crop_height <= image.shape[0]):
        x = random.randint(0, image.shape[1] - crop_width)
        y = random.randint(0, image.shape[0] - crop_height)

        if len(label.shape) == 3:
            return image[y:y + crop_height, x:x + crop_width, :], label[y:y + crop_height, x:x + crop_width, :]
        else:
            return image[y:y + crop_height, x:x + crop_width, :], label[y:y + crop_height, x:x + crop_width]
    else:
        raise Exception('Crop shape (%d, %d) exceeds image dimensions (%d, %d)!' % (
        crop_height, crop_width, image.shape[0], image.shape[1]))



def random_shadow(image):
    """
    Generates and adds random shadow
    """
    rand_width_scal_1 =  np.random.rand()
    IMAGE_HEIGHT, IMAGE_WIDTH,_ = image.shape
    x1, y1 = IMAGE_WIDTH *  rand_width_scal_1, 0
    rand_width_scal_2 =  np.random.rand()
    x2, y2 = IMAGE_WIDTH * rand_width_scal_2, IMAGE_HEIGHT
    xn, yn = np.mgrid[0:IMAGE_HEIGHT, 0:IMAGE_WIDTH]
    mask = np.zeros_like(image[:, :, 1])
    mask[(yn - y1) * (x2 - x1) - (y2 - y1) * (xn - x1) > 0] = 1

    cond = mask == np.random.randint(2)
    s_ratio = np.random.uniform(low=0.2, high=0.5)

    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    hls[:, :, 1][cond] = hls[:, :, 1][cond] * s_ratio
    return cv2.cvtColor(hls, cv2.COLOR_HLS2RGB)


def random_brightness(image):
    """
    Randomly adjust brightness of the image.
    HSV (Hue, Saturation, Value) is also called HSB ('B' for Brightness).
    """

    hsv_channel = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    brightness_scalar = np.random.rand()
    ratio = 1.0 + 0.4 * (brightness_scalar - 0.5)
    hsv_channel[:,:,2] =  hsv_channel[:,:,2] * ratio
    return cv2.cvtColor(hsv_channel, cv2.COLOR_HSV2RGB)


def bc_img(img, s=1.0, m=0.0):
    img = img.astype(np.int)
    img = img * s + m
    img[img > 255] = 255
    img[img < 0] = 0
    img = img.astype(np.uint8)
    return img

def data_augmentation(input_image, output_image):
    # Data augmentation
    # try:
    #     random_crop(image, label, 320, 240)
    # except:
    #     pass
    # go here
#     if random.randint(0,1):
#         input_image = cv2.flip(input_image, 1)
#         output_image = cv2.flip(output_image, 1)
#     if  random.randint(0,1):
#         input_image = cv2.flip(input_image, 0)
#         output_image = cv2.flip(output_image, 0)
    
    #brightness
    if random.randint(0,1):
        factor = 1.0 + random.uniform(-1.0*0.5, 0.5)
        table = np.array([((i / 255.0) * factor) * 255 for i in np.arange(0, 256)]).astype(np.uint8)
        input_image = cv2.LUT(input_image, table)
#     contr = random.uniform(0.85, 1.15) # Contrast augmentation
#     bright = random.randint(-40, 30) # Brightness augmentation
#     input_image = bc_img(input_image, contr, bright)
    
#     if random.randint(0,1):
#         input_image = cv2.flip(input_image, 1)
#         output_image = cv2.flip(output_image, 1)
#     if  random.randint(0,1):
#         input_image = cv2.flip(input_image, 0)
#         output_image = cv2.flip(output_image, 0)

#     contrast = random.uniform(0.8, 1)  # Contrast augmentation
#     bright = random.randint(-40, 30)  # Brightness augmentation
#     input_image = bc_img(input_image, contrast, bright)

#     if random.randint(0,1):
#         input_image = random_brightness(input_image)
        
    if random.randint(0,1):
        input_image = random_shadow(input_image)
#     if random.randint(0,1):
#         factor = 1.0 + random.uniform(-1.0*0.5, 0.5)
#         table = np.array([((i / 255.0) * factor) * 255 for i in np.arange(0, 256)]).astype(np.uint8)
#         input_image = cv2.LUT(input_image, table)
       
    #print('ok')
    
#     if  random.randint(0,1):
#         angle = random.uniform(-1*45, 45)
#         M = cv2.getRotationMatrix2D((input_image.shape[1]//2, input_image.shape[0]//2), angle, 1.0)
#         input_image = cv2.warpAffine(input_image, M, (input_image.shape[1], input_image.shape[0]), flags=cv2.INTER_NEAREST)
#         output_image = cv2.warpAffine(output_image, M, (output_image.shape[1], output_image.shape[0]), flags=cv2.INTER_NEAREST)

    return input_image, output_image


def gen_batch_function(samplesX, samplesY, label_values, batch_size=1, is_train=True):
    """
    F function to create batches of training data
    :param data_folder: Path to folder that contains all the datasets
    :param image_shape: Tuple - Shape of image
    :return:"""

    samplesX, samplesY = shuffle(samplesX, samplesY)

    # Shuffle training data

    num_samples = len(samplesX)

    if(batch_size==-1):
        batch_size = num_samples

    # Loop through batches and grab images, yielding each batch
    for batch_i in range(0, num_samples, batch_size):

        X_train = samplesX[batch_i:batch_i + batch_size]
        y_train = samplesY[batch_i:batch_i + batch_size]
        
        # preprocessing if required
        X_f = []
        y_f = []
        for x, y in zip(X_train, y_train):
            if is_train:
                x, y = data_augmentation(x, y)
            y = np.float32(one_hot_it(y, label_values=label_values))
            X_f.append(x)
            y_f.append(y)

        X_f = np.float32(X_f)
        y_f = np.float32(y_f)
        yield X_f, y_f


In [44]:

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'

    graph = tf.get_default_graph()
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3 = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4 = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7 = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    return input, keep_prob, layer3, layer4, layer7


def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    layer7_conv_1x1 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, 1,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                       kernel_regularizer=tf.contrib.layers.l2_regularizer(L2_REG), name='conv_1_1_1',activation = tf.nn.relu)

   # layer7_conv_1x1 = tf.layers.batch_normalization(layer7_conv_1x1)

    output = tf.layers.conv2d_transpose(layer7_conv_1x1, num_classes, 4, 2,
                                        padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                        kernel_regularizer=tf.contrib.layers.l2_regularizer(L2_REG), name='conv_1_1_2',activation = tf.nn.relu)
    #output = keras.layers.UpSampling2D(size=(2,2),data_format=None,interpolation='bilinear')(layer7_conv_1x1)

    layer4_conv_1x1 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, 1,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                       kernel_regularizer=tf.contrib.layers.l2_regularizer(L2_REG), name='conv_1_1_3',activation = tf.nn.relu)
    #layer4_conv_1x1 = tf.layers.batch_normalization(layer4_conv_1x1)

    output = tf.add(output, layer4_conv_1x1, name='conv_1_1_4')
    output = tf.layers.batch_normalization(output)

    #output = keras.layers.UpSampling2D(size=(2,2),data_format=None,interpolation='bilinear')(output)


    output = tf.layers.conv2d_transpose(output, num_classes, 4, 2,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                        kernel_regularizer=tf.contrib.layers.l2_regularizer(L2_REG),  name='conv_1_1_5',activation = tf.nn.relu)
    layer3_conv_1x1 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, 1,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                       kernel_regularizer=tf.contrib.layers.l2_regularizer(L2_REG), name='conv_1_1_6',activation = tf.nn.relu)
    output = tf.add(output, layer3_conv_1x1,  name='conv_1_1_7')
    output = tf.layers.batch_normalization(output)

    #output = keras.layers.UpSampling2D(size=(8,8),data_format=None,interpolation='bilinear')(output)
    output = tf.layers.conv2d_transpose(output, num_classes, 16, 8,
                                        padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                        kernel_regularizer=tf.contrib.layers.l2_regularizer(L2_REG),  name='conv_1_1_8',activation = tf.nn.relu)

    return output



def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function

    logits = tf.reshape(nn_last_layer, (-1, num_classes), name='logits')
    labels = tf.reshape(correct_label, (-1, num_classes), name='labels')
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    softmax = tf.nn.softmax(logits, name='softmax')
    
   # global_step = tf.Variable(0, trainable=False)

#     train_op = tf.train.AdamOptimizer(learning_rate=
#                                       cyclic_learning_rate(global_step=global_step,
#                                                                mode='triangular2')).minimize(cross_entropy_loss,global_step=global_step)
    train_op = tf.train.AdamOptimizer(learning_rate=0.0005).minimize(cross_entropy_loss)

#     gradients = optimizer.compute_gradients(cross_entropy_loss)
#     capped_gvs = [(tf.clip_by_value(grad, -.5, .5), var) for grad, var in gradients]
#     train_op = optimizer.apply_gradients(capped_gvs)

    return logits, train_op, cross_entropy_loss



def train_nn_sky(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate, X_train, y_train, label_values, X_val, y_val):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    image_shape = (160, 576)  # KITTI dataset uses 160x576 images
    data_dir = './data'
    runs_dir = './runs'

   
    for epoch in range(epochs):
        print('epoch : ', epoch)
        for image, targets in get_batches_fn(X_train, y_train, label_values, batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss],
                               feed_dict={input_image: image, correct_label: targets, keep_prob: KEEP_PROB,
                                          learning_rate:LEARNING_RATE})
#         assign_op = global_step.assign(epoch)
#         sess.run(assign_op)
        # Print data on the learning process

            print("Epoch: {}".format(epoch + 1), "/ {}".format(epochs), " Loss: {:.3f}".format(loss))
        
        mean_loss = []
        for image, targets in get_batches_fn(X_val, y_val, label_values, batch_size, is_train=False):
            loss = sess.run([cross_entropy_loss],
                               feed_dict={input_image: image, correct_label: targets, keep_prob:1})
            mean_loss.append(loss)

        mean_loss_ = np.mean(np.array(mean_loss))
        print("Epoch: {}".format(epoch + 1), "/ {}".format(epochs), " Validation Loss: {:.3f}".format(mean_loss_))
        
        print('saving model')
        saver.save(sess, './model')

        # validate_nn(sess, epochs, 1, get_batches_fn_valid, train_op, cross_entropy_loss, input, correct_label,
        #             keep_prob, learning_rate)



TensorFlow Version: 1.12.0
Default GPU Device: /device:GPU:0


In [45]:
print(label_values)

[(0, 0, 0), (111, 74, 0), (81, 0, 81), (128, 64, 128), (244, 35, 232), (250, 170, 160), (230, 150, 140), (70, 70, 70), (102, 102, 156), (190, 153, 153), (180, 165, 180), (150, 100, 100), (150, 120, 90), (153, 153, 153), (250, 170, 30), (220, 220, 0), (107, 142, 35), (152, 251, 152), (70, 130, 180), (220, 20, 60), (255, 0, 0), (0, 0, 142), (0, 0, 70), (0, 60, 100), (0, 0, 90), (0, 0, 110), (0, 80, 100), (0, 0, 230), (119, 11, 32)]


In [46]:
num_classes = 29
image_shape = (160, 576)  # KITTI dataset uses 160x576 images
data_dir = './data'
runs_dir = './runs'
tests.test_for_kitti_dataset(data_dir)
epochs = 30
batch_size = 25
# Download pretrained vgg model
helper.maybe_download_pretrained_vgg(data_dir)

# OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
# You'll need a GPU with at least 10 teraFLOPS to train on.
#  https://www.cityscapes-dataset.com/

#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
print('run')


tf.reset_default_graph()
with tf.Session() as sess:

    # Path to vgg model
    vgg_path = os.path.join(data_dir, 'vgg')
    # Create function to get batches
    get_batches_fn = gen_batch_function

    input, keep_prob, layer3, layer4, layer7 = load_vgg(sess, vgg_path)
    print(input)
    output = layers(layer3, layer4, layer7, num_classes)
    correct_label = tf.placeholder(dtype=tf.float32, shape=(None, None, None, num_classes))
    learning_rate = tf.placeholder(dtype=tf.float32)
    logits, train_op, cross_entropy_loss = optimize(output, correct_label, learning_rate, num_classes)
    sess.run(tf.global_variables_initializer())

    saver = tf.train.Saver()  # Simple model saver

    #saver.restore(sess, tf.train.latest_checkpoint('.'))

    train_nn_sky(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input, correct_label,
             keep_prob, learning_rate, X_train, y_train, label_values, X_val, y_val)


    # Save inference data using helper.save_inference_samples
    #helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input)
    # OPTIONAL: Augment Images for better results
    #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

    # TODO: Build NN using load_vgg, layers, and optimize function

    # TODO: Train NN using the train_nn function

    # TODO: Save inference data using helper.save_inference_samples
    #  helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

    # OPTIONAL: Apply the trained model to a video




test_1
test_1
Tests Passed
run
INFO:tensorflow:Restoring parameters from ./data\vgg\variables\variables
Tensor("image_input:0", shape=(?, ?, ?, 3), dtype=float32)
epoch :  0
Epoch: 1 / 30  Loss: 10.173
Epoch: 1 / 30  Loss: 3.702
Epoch: 1 / 30  Loss: 3.310
Epoch: 1 / 30  Loss: 3.281
Epoch: 1 / 30  Loss: 3.159
Epoch: 1 / 30  Loss: 3.444
Epoch: 1 / 30  Loss: 3.008
Epoch: 1 / 30  Loss: 3.050
Epoch: 1 / 30  Loss: 2.950
Epoch: 1 / 30  Loss: 2.862
Epoch: 1 / 30  Loss: 2.679
Epoch: 1 / 30  Loss: 2.720
Epoch: 1 / 30  Loss: 2.586
Epoch: 1 / 30  Loss: 2.523
Epoch: 1 / 30  Loss: 2.456
Epoch: 1 / 30  Loss: 2.515
Epoch: 1 / 30  Loss: 2.378
Epoch: 1 / 30  Loss: 2.284
Epoch: 1 / 30  Loss: 2.301
Epoch: 1 / 30  Loss: 2.279
Epoch: 1 / 30  Loss: 2.358
Epoch: 1 / 30  Loss: 2.302
Epoch: 1 / 30  Loss: 2.230
Epoch: 1 / 30  Loss: 2.195
Epoch: 1 / 30  Loss: 2.258
Epoch: 1 / 30  Loss: 2.359
Epoch: 1 / 30  Loss: 2.174
Epoch: 1 / 30  Loss: 2.300
Epoch: 1 / 30  Loss: 2.065
Epoch: 1 / 30  Loss: 2.184
Epoch: 1 / 30  

KeyboardInterrupt: 

In [ ]:

sess = tf.Session()
saver = tf.train.Saver()  # Simple model saver

saver.restore(sess, tf.train.latest_checkpoint('.'))


vgg_input_tensor_name = 'image_input:0'
vgg_keep_prob_tensor_name = 'keep_prob:0'


graph = tf.get_default_graph()

input = graph.get_tensor_by_name(vgg_input_tensor_name)
keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)


In [ ]:
print(label_values)

c_values = label_values

print(c_values)

In [ ]:
def pipeline_final(img):
    img= cv2.resize(img, dsize=(512, 256))
    print(img.shape,'img_shape')
    img = np.array([img])
    logits = tf.reshape(output, (-1, num_classes))
    softmax = tf.nn.softmax(logits, name='softmax')



    softmax_ = loss = sess.run([softmax],
                               feed_dict={input: img, keep_prob:1})

    logits_ = (softmax_[0].reshape(1,256,512,29)) #reshape(256,512) = rezie(512,256) ! fuck :S
    print(logits_[0].shape, 'fin')
    output_image = cityscapes_helper.reverse_one_hot(logits_[0])
    out_vis_image = cityscapes_helper.colour_code_segmentation(output_image, c_values)
    
    a = cv2.cvtColor(np.uint8(out_vis_image),4)
    print(a.shape)
    b=cv2.cvtColor(np.uint8(img[0]),4)
    print(b.shape)
    added_image = cv2.addWeighted(a,1,b,1,4)
    added_image= cv2.resize(added_image, dsize=(512, 256))
    print(added_image.shape,'img added')
    cv2.imwrite('combined.png', added_image)
    return added_image

In [ ]:
#input, keep_prob, layer3, layer4, layer7 = load_vgg(sess, vgg_path)
import cv2
def load_image(path):
    image = cv2.cvtColor(cv2.imread(path,-1), cv2.COLOR_BGR2RGB)
    return image

In [ ]:

image = load_image('tt2.png')

In [ ]:
pipeline_final(image)

In [93]:
from moviepy.editor import VideoFileClip
data_dir = './data'
num_classes = 29
tf.reset_default_graph()
sess = tf.Session()

# Path to vgg model
vgg_path = os.path.join(data_dir, 'vgg')
# Create function to get batches
get_batches_fn = gen_batch_function

input, keep_prob, layer3, layer4, layer7 = load_vgg(sess, vgg_path)
print(input)
output = layers(layer3, layer4, layer7, num_classes)
correct_label = tf.placeholder(dtype=tf.float32, shape=(None, None, None, num_classes))
learning_rate = tf.placeholder(dtype=tf.float32)
logits, train_op, cross_entropy_loss = optimize(output, correct_label, learning_rate, num_classes)
tf.set_random_seed(123)
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()  # Simple model saver

saver.restore(sess, tf.train.latest_checkpoint('.'))

logits = tf.reshape(output, (-1, num_classes))
softmax = tf.nn.softmax(logits, name='softmax')

def pipeline_final(img, is_video):
    channel = 1 if is_video else 4
    size = img.shape
    img= cv2.resize(img, dsize=(512, 256))
    img = np.array([img])

    softmax_ = loss = sess.run([softmax],
                       feed_dict={input: img, keep_prob:1})
    logits_ = (softmax_[0].reshape(1,256,512,29)) #reshape(256,512) = rezie(512,256) ! fuck :S
    output_image = reverse_one_hot(logits_[0])

    print(output_image.shape)

    out_vis_image = colour_code_segmentation(output_image, c_values)

    a = cv2.cvtColor(np.uint8(out_vis_image), channel)

    b = cv2.cvtColor(np.uint8(img[0]), channel)

    added_image = cv2.addWeighted(a, 1, b, 1, channel)
    added_image = cv2.resize(added_image, dsize=(512,256))

    return added_image

def pipeline_video(img):
    return pipeline_final(img, True)

def pipeline_img(img):
    return pipeline_final(img, False)

def process(media_dir, save_dir, model_dir=''):

    try:
        img = load_image(media_dir)
        output = os.path.join(save_dir, 'output_image.png')
        img = pipeline_img(img)
        cv2.imwrite(output, img)
    except Exception as ex:
        output = os.path.join(save_dir, 'output_video2.mp4')
        clip1 = VideoFileClip(media_dir)
        white_clip = clip1.fl_image(pipeline_video)

        white_clip.write_videofile(output, audio=False)

INFO:tensorflow:Restoring parameters from ./data\vgg\variables\variables
Tensor("image_input:0", shape=(?, ?, ?, 3), dtype=float32)
INFO:tensorflow:Restoring parameters from .\model


In [94]:
image = process('2.mp4', '')

(256, 512)
[MoviePy] >>>> Building video output_video2.mp4
[MoviePy] Writing video output_video2.mp4


  0%|                                                                                         | 0/2101 [00:00<?, ?it/s]

(256, 512)
(256, 512)


  0%|                                                                                 | 2/2101 [00:00<02:44, 12.78it/s]

(256, 512)
(256, 512)


...

(256, 512)
(256, 512)


100%|█████████████████████████████████████████████████████████████████████████████▉| 2100/2101 [02:51<00:00, 12.49it/s]

(256, 512)


100%|██████████████████████████████████████████████████████████████████████████████| 2101/2101 [02:51<00:00, 12.25it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_video2.mp4 



In [39]:
import os
data_dir = 'D:\data\leftImg8bit_trainvaltest'
from glob import glob



c = os.path.join(data_dir, 'leftImg8bit')

print(c)

D:\data\leftImg8bit_trainvaltest\leftImg8bit


In [41]:
print(c)

a = glob(os.path.join(c, 'test/*/', '*.png'))

D:\data\leftImg8bit_trainvaltest\leftImg8bit


In [42]:
print(a)

['D:\\data\\leftImg8bit_trainvaltest\\leftImg8bit\\test\\berlin\\berlin_000000_000019_leftImg8bit.png', 'D:\\data\\leftImg8bit_trainvaltest\\leftImg8bit\\test\\berlin\\berlin_000001_000019_leftImg8bit.png', 'D:\\data\\leftImg8bit_trainvaltest\\leftImg8bit\\test\\berlin\\berlin_000002_000019_leftImg8bit.png', 'D:\\data\\leftImg8bit_trainvaltest\\leftImg8bit\\test\\berlin\\berlin_000003_000019_leftImg8bit.png', 'D:\\data\\leftImg8bit_trainvaltest\\leftImg8bit\\test\\berlin\\berlin_000004_000019_leftImg8bit.png', 'D:\\data\\leftImg8bit_trainvaltest\\leftImg8bit\\test\\berlin\\berlin_000005_000019_leftImg8bit.png', 'D:\\data\\leftImg8bit_trainvaltest\\leftImg8bit\\test\\berlin\\berlin_000006_000019_leftImg8bit.png', 'D:\\data\\leftImg8bit_trainvaltest\\leftImg8bit\\test\\berlin\\berlin_000007_000019_leftImg8bit.png', 'D:\\data\\leftImg8bit_trainvaltest\\leftImg8bit\\test\\berlin\\berlin_000008_000019_leftImg8bit.png', 'D:\\data\\leftImg8bit_trainvaltest\\leftImg8bit\\test\\berlin\\berlin_0